In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.2.1-bin-hadoop3.2'

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [5]:
df = spark.read.format("json").option("multiLine", "true").load("reviews.json")

In [9]:
review = df.select('paper.review').collect()[0][0]


In [13]:
confidence_col = []
evaluation_col = []
orientation_col = []
timespan_col= []
for i in review:
    for j in i:
        confidence_col.append(j[0])
        evaluation_col.append(j[1])
        orientation_col.append(j[4])
        timespan_col.append(j[7])



405 405 405 405


In [14]:
import pandas as pd
from pyspark.sql import SQLContext

# intialise data of lists.
data = {'confidence':confidence_col,
       'evaluation':evaluation_col,'orientation':orientation_col,'timespan':timespan_col}
  
# Create DataFrame
review_data = pd.DataFrame(data)
review_data = review_data.dropna()
review_data ['confidence']= review_data[['confidence']].astype(int)
review_data ['orientation']= review_data[['orientation']].astype(int)
review_data ['evaluation']= review_data[['evaluation']].astype(int)

review_data = spark.createDataFrame(data=review_data)
review_data.show()

+----------+----------+-----------+----------+
|confidence|evaluation|orientation|  timespan|
+----------+----------+-----------+----------+
|         4|         1|          0|2010-07-05|
|         4|         1|          1|2010-07-05|
|         5|         1|          1|2010-07-05|
|         4|         2|          1|2010-07-05|
|         4|         2|          0|2010-07-05|
|         4|         2|          0|2010-07-05|
|         4|         2|          1|2010-07-05|
|         3|         2|          1|2010-07-05|
|         3|         0|         -1|2010-07-05|
|         4|         2|          2|2010-07-05|
|         2|        -2|         -1|2010-07-05|
|         4|         2|          1|2010-07-05|
|         4|         2|          0|2010-07-05|
|         5|         2|          1|2010-07-05|
|         4|        -1|          0|2010-07-05|
|         4|        -2|         -1|2010-07-05|
|         4|         1|         -1|2010-07-05|
|         5|        -2|         -1|2010-07-05|
|         4| 

# Sampling/filtering RDDs to pick out relevant data points


In [18]:
from time import time

In [26]:
sampled = review_data.sample(False, 0.1, 42)

contains_date_sample = sampled.filter(sampled.timespan =='2010')
t0 = time()
num_sampled = contains_date_sample.count()
duration = time() - t0
duration

2.9679691791534424

In [27]:

contains_date_all = review_data.filter(review_data.timespan =='2010')
t0 = time()
num_sampled = contains_date_sample.count()
duration = time() - t0
duration

3.1085140705108643

In [33]:
date_sample = sampled.filter(sampled.timespan =='2010')
non_date_sample = sampled.subtract(date_sample)

sampled.count()



45

In [34]:
date_sample.count()



0

In [35]:
non_date_sample.count()

36

In [42]:
feature_1 = sampled.rdd.map(lambda features: features[1]).distinct()
feature_2 = sampled.rdd.map(lambda features: features[3]).distinct()
f1 = feature_1.collect()
f2 = feature_2.collect()

In [43]:
f1

[0, 1, 2, -2, -1]

In [44]:
f2

['2010-07-05', '2015-07-05', '2014-07-05', '2013-07-05']